In [ ]:
import coco_dataset
import pathlib
import show_utils
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.transforms.functional as F
import torch
import common
import numpy as np
import copy

In [ ]:
inp_size = (368, 368)
targ_size = (46, 46)
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "train2017"),
    annFile=str(pathlib.Path("../coco") / "annotations" / "annotations" / "person_keypoints_train2017.json"),
    input_transform=transforms.Compose([
        transforms.Resize(inp_size),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    targ_size=targ_size)
i = 0

In [ ]:
i = 5

In [ ]:
tf_image, pafs, heatmaps, image, paf_locs, target = coco_dataset[i] # ,mask_out

show_utils.show_coco(image, target, coco_dataset.coco, draw_bbox=False)

print(f"sample: {i}")
print(image.size)
i += 1

In [ ]:
import post
kpt_groups = post.post_process(heatmaps, pafs, image.size)

if kpt_groups:
    keypoints_post = post.coco_format(kpt_groups)
    show_utils.draw_keypoints(copy.deepcopy(image), keypoints_post, connectivity=common.connect_skeleton, keypoint_color="orange", line_color="magenta")
else:
    print("NO DETECTIONS")

In [ ]:
heatmaps = heatmaps.numpy()
pafs = pafs.numpy()
paf_locs = np.array(paf_locs, dtype=float)

# mask_out = mask_out.squeeze_(0).numpy()

In [ ]:
#---------heatmaps---------
print(f"Heatmaps.size = {heatmaps.shape}")
show_utils.plot_grid(heatmaps, 2, 10, (17,4))

#---------pafs---------
print(f"Pafs.size = {pafs.shape}")
show_utils.plot_grid(pafs[[x for x in range(len(pafs)) if x%2==0]], 1, 17, (17, 2))
show_utils.plot_grid(pafs[[x for x in range(len(pafs)) if x%2==1]], 1, 17, (17, 2))

#---------blend---------
show_utils.blend(heatmaps, image, rows=7, cols=3, figsize=(15, 30))
show_utils.blend(paf_locs, image, rows=7, cols=3, figsize=(15, 30))
# show_utils.blend([mask_out], image, rows=1, cols=2, figsize=(12, 4))

#---------surface---------
# show_utils.surface(heatmaps[5])

# show_utils.show_pafs_quiver(pafs, keypoints, size=targ_size)
# plt.figure()
# show_utils.show_pafs_quiver_combined(pafs, size=targ_size)